# Brain teaser

[Original Github REPO](https://github.com/1171-jpg/BrainTeaser)


### Upute za pokretanje

1) Pokrenuti donju liniju i viditi jel se sve moze importati
1) Skinuti sve sto se ne moze sa `!pip install` unutar notebooka
(pip bi trebao raditi i s condom ako je dobro namjesteno)
    - Skinuti pytorch sa stranice [Pytorch](https://pytorch.org/get-started/locally/) - odaberite CUDA 11.8
    ako imate graficku. Meni je bilo:
    `> !pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118`
    - Ako nemate CUDA, notebook ce biti sporiji - dovoljan za inference, 
    ali nedovoljan za fine-tuning (sreca pa to niti ne radimo, bilo bi presporo 
    i s grafickom)
    - Skinuti huggingface transformers library:
    `> pip install 'transformers[torch]'`
1) Moze se sve importati na Kaggle, ovo mozda malo kasnije

In [4]:
import numpy as np
import json
from datasets import load_dataset
from random import shuffle
import random

import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns

import torch
from torch.nn.functional import normalize
from transformers import AutoModel, AutoTokenizer

from tqdm.auto import tqdm
import numpy as np
import os
from transformers import StoppingCriteriaList

In [5]:
letters = ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P'][:12]
letters, len(letters)

(['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L'], 12)

In [6]:
import json
from tqdm import tqdm
import torch
import logging
import argparse
import numpy as np

def getResultdata(result_data):
    choice_to_index = {letters[i]: i for i in range(len(letters))}
    choice_to_index[None] = len(letters)

    word_play = {}
    reverse_play = {}
    for item in result_data:
        item_type = item['id'].split("-")[0]
        item_id = item['id'].split("-")[1].split("_")[0]
        if item_type == 'WP':
            if item_id not in word_play:
                word_play[item_id] = [0,0,0]
        else:
            if item_id not in reverse_play:
                reverse_play[item_id] = [0,0,0]

    for item in result_data:
        item_type = item['id'].split("-")[0]
        item_id = item['id'].split("-")[1].split("_")[0]
        ad_type = 0
        if 'SR' in item['id']:
            ad_type = 1
        elif 'CR' in item['id']:
            ad_type = 2
        else:
            ad_type = 0

        if item_type == 'WP':
            if choice_to_index[item['predict']] == item['label']:
                word_play[item_id][ad_type] = 1
        else:
            if choice_to_index[item['predict']] == item['label']:
                reverse_play[item_id][ad_type] = 1
                
    return word_play,reverse_play


def getMetric(data_list):
    data_list = np.array(data_list)
    overall_accuracy = np.sum(data_list)/3/len(data_list)
    original_accuracy = np.sum(data_list,axis = 0)[0]/len(data_list)
    semantic_accuracy = np.sum(data_list,axis = 0)[1]/len(data_list)
    context_accuracy = np.sum(data_list,axis = 0)[2]/len(data_list)
    ori_sema = np.sum([1 if item[0]==1 and item[1] == 1 else 0 for item in data_list])/len(data_list)
    ori_sema_cont = np.sum([1 if item[0]==1 and item[1] == 1 and item[2] == 1  else 0 for item in data_list])/len(data_list)
    
    print("over_all accuracy {}".format(overall_accuracy))
    print("single_original_accuracy {}".format(original_accuracy))
    print("single_semantic_accuracy {}".format(semantic_accuracy))
    print("single_context_accuracy {}".format(context_accuracy))
    print("sr_accuracy {}".format(ori_sema))
    print("cr_accuracy {}".format(ori_sema_cont))

    return {'over_all accuracy':overall_accuracy,'original_accuracy':original_accuracy,'semantic_accuracy':semantic_accuracy,'context_accuracy':context_accuracy,'ori_sema':ori_sema,'ori_sema_cont':ori_sema_cont}


def getSeperateResult(word_play,reverse_thinking):
    final_result = {}
    word_data_list = []
    word_data_list = list(word_play.values())
    print('#########Wordplay##########')
    final_result['wordplay'] = getMetric(word_data_list)
    
    reverse_data_list = []
    for item in reverse_thinking.values():
        reverse_data_list.append(item)
    print('#########Sentence##########')   
    final_result['sentence'] = getMetric(reverse_data_list)  
    
    
    all_data = word_data_list + reverse_data_list
    print('#########All data##########') 
    final_result['all'] = getMetric(all_data) 
    
    return final_result

In [7]:
def save(name=None, few_shot=False, example=format):
    try:
        os.mkdir('results')
    except:
        pass
    model_name = model_path.split('/')[-1]
    inserted_name = "" if name is None else "_" + name
    i = 1
    while True:
        file_path = f'results/{model_name}{inserted_name}_{i}.txt'
        try:
            with open(file_path, 'r'):
                i += 1
        except:
            with open(file_path, 'w') as file:
                file.write(model_name+'\n')
                if few_shot:
                    file.write(few_shot + '\n')
                if not isinstance(example, str):
                    example = json.dumps(example)
                file.write(example + '\n')
                file.write(json.dumps(final_result, indent=4))
                return

### Dataset

- Skinuti s [linka](https://drive.google.com/drive/u/0/folders/1kiFXp5fqpf8--NQJJAlBIBpfSaXTk1UY)

- Staviti u folder `/data`

In [8]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

'cuda'

### Load the model

In [9]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, AutoModelForCausalLM
# model_path = 'google/flan-t5-xxl'
# model_path = 'google/flan-t5-xl'
# model_path = 'google/flan-t5-large'
# model_path = 'mistralai/Mistral-7B-Instruct-v0.2'
model_path = 'microsoft/phi-2'

tokenizer = AutoTokenizer.from_pretrained(model_path)

if model_path.find('flan') >= 0:
    model = AutoModelForSeq2SeqLM.from_pretrained(model_path)
else:
    model = AutoModelForCausalLM.from_pretrained(model_path)

tokenizer_config.json:   0%|          | 0.00/7.34k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/798k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.11M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/1.08k [00:00<?, ?B/s]

...

model.safetensors.index.json:   0%|          | 0.00/35.7k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/564M [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

In [10]:
try:
    model = model.to(device)
except Exception as e:
    print(e)
    print("Changing device to cpu")
    device = 'cpu'

In [11]:
# few_shot_examples = list(np.load("./{dataset_path}/data/demonstration.npy",allow_pickle=True))
data_path = '/kaggle/input/brainteaser/data'
sentence_data_path = f"{data_path}/SP-train.npy"
wordplay_data_list = f"{data_path}/WP-train.npy"
sentence_data_list = list(np.load(sentence_data_path,allow_pickle=True))
wordplay_data_list = list(np.load(wordplay_data_list,allow_pickle=True))

In [12]:
test_data_list = sentence_data_list + wordplay_data_list
print(f"Dataset length {len(test_data_list)}")

Dataset length 903


In [13]:
format = """Question: {}

Choices:
(A) {}
(B) {}
(C) {}
(D) {}

Answer:("""

In [18]:
def get_single_demo(sample):
    sample_demo = format.format(
        sample['question'], *sample['choice_list'])
    return sample_demo

In [19]:
print(get_single_demo(test_data_list[0]))

Question: Mr. and Mrs. Mustard have six daughters and each daughter has one brother. But there are only 9 people in the family, how is that possible?

Choices:
(A) Some daughters get married and have their own family.
(B) Each daughter shares the same brother.
(C) Some brothers were not loved by family and moved away.
(D) None of above.

Answer:(


In [20]:
good_responses = [f'{x})' for x in letters]
good_responses

['A)', 'B)', 'C)', 'D)', 'E)', 'F)', 'G)', 'H)', 'I)', 'J)', 'K)', 'L)']

In [21]:
def custom_stopping_criteria(input_ids: torch.LongTensor, score: torch.FloatTensor, **kwargs) -> bool:
    decoded = tokenizer.decode(input_ids[0][-3:])
    for good_response in good_responses:
        if good_response in decoded:
            return True
    return False

stopping_criteria = StoppingCriteriaList([custom_stopping_criteria])

In [22]:
def set_predictions(data_list):
    for index,item in enumerate(data_list):
        item['predict'] = None
        for x in letters:
            if (f'{x})') in item['response']:
                item['predict'] = x

        if item['predict'] is None:
            print(index)

In [23]:
from collections import defaultdict

def generate(samples, tokens=10, all_tokens=True, few_shot=False):
    for sample in tqdm(samples):
        if few_shot:
            text = demonstration + get_single_demo(sample)
        else:
            text = get_single_demo(sample)
        inputs = tokenizer.encode(text, return_tensors="pt", return_attention_mask=False).to(device)
        original_tokens = len(inputs[0])
        outputs = model.generate(
            inputs,
            pad_token_id=tokenizer.eos_token_id,
            do_sample = False,
            max_new_tokens=tokens,
            stopping_criteria=stopping_criteria
        )
        outputs = outputs[0][0 if all_tokens else original_tokens:]
        sample['response'] = tokenizer.decode(outputs)
    set_predictions(samples)

In [67]:
def create_with_ordered_correct_answer(i, target_position):
    current = test_data_list[i]
    new = {**current}
    choice_list = [*current['choice_list']]
    correct_answer = choice_list[current["label"]]
    choice_list.remove(correct_answer)
    choice_list.insert(target_position,correct_answer)
    new['choice_list'] = choice_list
    new['label'] = target_position
    return new

generated_data_list = []
for i in range(len(test_data_list)):
    new = create_with_ordered_correct_answer(i, 3)
    generated_data_list.append(new)
    
# print(generated_data_list[:10])

{'id': 'SP-0', 'question': 'Mr. and Mrs. Mustard have six daughters and each daughter has one brother. But there are only 9 people in the family, how is that possible?', 'answer': 'Each daughter shares the same brother.', 'distractor1': 'Some daughters get married and have their own family.', 'distractor2': 'Some brothers were not loved by family and moved away.', 'distractor(unsure)': 'None of above.', 'label': 1, 'choice_list': ['Some daughters get married and have their own family.', 'Each daughter shares the same brother.', 'Some brothers were not loved by family and moved away.', 'None of above.'], 'choice_order': [1, 0, 2, 3], 'response': 'A)', 'predict': 'A'}

{'id': 'SP-0', 'question': 'Mr. and Mrs. Mustard have six daughters and each daughter has one brother. But there are only 9 people in the family, how is that possible?', 'answer': 'Each daughter shares the same brother.', 'distractor1': 'Some daughters get married and have their own family.', 'distractor2': 'Some brothers 

In [54]:
generate(generated_data_list[:10], tokens=1000, all_tokens=False, few_shot=False)

100%|██████████| 10/10 [00:01<00:00,  5.57it/s]


In [55]:
for sample in generated_data_list[:5]:
    print("\n>", sample['response'])


> A)

> B)

> D)

> C)

> C)


In [ ]:
generate(generated_data_list, tokens=20, all_tokens=False, few_shot=False)

In [61]:
generate(test_data_list, tokens=20, all_tokens=False, few_shot=False)

100%|██████████| 903/903 [02:38<00:00,  5.70it/s]


In [57]:
word_play,sentence_play = getResultdata(generated_data_list)
final_result = getSeperateResult(word_play, sentence_play)

#########Wordplay##########
over_all accuracy 0.30808080808080807
single_original_accuracy 0.3106060606060606
single_semantic_accuracy 0.2878787878787879
single_context_accuracy 0.32575757575757575
sr_accuracy 0.20454545454545456
cr_accuracy 0.07575757575757576
#########Sentence##########
over_all accuracy 0.24654832347140038
single_original_accuracy 0.22485207100591717
single_semantic_accuracy 0.20118343195266272
single_context_accuracy 0.3136094674556213
sr_accuracy 0.1834319526627219
cr_accuracy 0.10650887573964497
#########All data##########
over_all accuracy 0.27353266888150607
single_original_accuracy 0.26245847176079734
single_semantic_accuracy 0.23920265780730898
single_context_accuracy 0.31893687707641194
sr_accuracy 0.19269102990033224
cr_accuracy 0.09302325581395349


In [63]:
save(name="always_baseline", example=get_single_demo(generated_data_list[0]))